In [12]:
''' Importar dependencias'''
import matplotlib.pyplot as plt
import numpy as np

In [5]:
#cargar el texto
text = open("secret.txt",'r').read()

# Preprocesamiento del texto
Antes que nada, para un mejor manejo del texto, nos aseguramos que no tenga ningún caracter. 
En este caso simplemente quitamos puntos ("."), comas (",") o espacios. Esta funcionalidad podría extenderse a 
cualquier caracter

In [2]:
def preprocess_text(text):
    new_text = text.replace(".","")
    new_text = new_text.replace(",","")
    new_text = new_text.replace(" ","")	
    new_text = new_text.replace("\n","")
    return new_text

# Encontrar bigramas y trigramas
Generalmente lo primero que se hace es encontrar bigramas y trigramas en el texto cifrado, 
en este caso se define una función que calcula los *n-gramas* que básicamente consiste en subconjuntos de 
$N$ caracteres, el caso para $N = 2$ es un bigrama y $N = 3$ es un trigrama.
La función devuelve una diccionario de n-gramas, cada uno con una lista de posiciones donde se contraron en el texto.
Nótese que los n-gramas que solo se encontraron una vez se omiten. Por ejemplo, un posible resultado sería

$\{ GA : [1,15], AB : [4,18,140] \}$

Opcionalmente, para reducir el espacio de búsqueda en funciones posteriores, se indica como parámetro de la función un límite de n-gramas a regresar (*top*). Por ejemplo, si `top=10`. La función solamente regresa
el top 10 de n-gramas con mayor frecuencia en el texto cifrado. Si se llama con `top=0`, la función regresa todos
los n-gramas del texto.


In [6]:
def calculate_n_grams(n,text,top=0):
    ''' 
    n_grams = {'key' : [position_0,position_1 ... ]}
    ''' 
    n_grams = {}
    ''' Calcula todos los n-gramas en el texto''' 
    for i in range(0,len(text)):
        n_gram = text[i:i+n]
        if n_gram not in n_grams:
            n_grams[n_gram] = [i]
        else:
            n_grams[n_gram].append(i)
    '''Elimina todos los n-gramas cuyas ocurrencias sean 1.
    Recordar que value es una lista de posiciones.'''
    n_grams_most_ocurrences = {}
    for key,value in n_grams.items():
        if len(value) > 1:
            n_grams_most_ocurrences[key] = value
    if top > 0:
        n_grams_most_ocurrences = sorted(n_grams_most_ocurrences.items(), key=lambda kv: len(kv[1]),reverse=True)
        n_grams_most_ocurrences = dict(n_grams_most_ocurrences[:top]) 
    return n_grams_most_ocurrences

# Calcular histograma de frecuencias

Una herramienta muy poderosa en el criptoanálisis es el análisis de frecuencia de caractéres
(o n-gramas) de un texto cifrado. A continuación se define la función `frequency_histogram(dictionary)` que recibe
como parámetro un diccionario de tipo `{n-gram:[positions]}`. 
Nótese que esta funcion está hecha para recibir un diccionario exactamente como lo regresa la 
función anterior `calculate_n_grams()`. Si se quisiera realizar un análisis de frecuencia para cada caracter
del texto, simplemente se debe obtener una lista llamando a la funcion `calculate_n_grams()` con un parámetro $N=1$. 

In [30]:
def frequency_histogram(dictionary):
    width_bar = 0.5
    fig,ax = plt.subplots()
    tick_labels = tuple([key for key,_ in dictionary.items()])
    frequencies = [len(value) for _,value in dictionary.items()]
    ind = np.arange(len(frequencies))
    ax.set_title("Frecuency Histogram")
    ax.bar(ind,frequencies,width_bar,color='b')
    ax.set_ylabel('Frenquencies')
    ax.set_xlabel('N-grams')
    ax.set_xticks(ind)
    ax.set_xticklabels(tick_labels)
    plt.show()

# Calcular diferencias entre las posiciones
El siguiente paso es *intentar* encontrar la longitud de la clave, esto se realiza calculando todas las diferencias entre las posiciones de cada n-grama. Por ejemplo si un bigrama **GD** aparece en las posiciones 
$\{1,31,144 \}$ del texto cifrado, entonces las posibles longitudes de la clave serán $\{31-1 = 30,144-1=143,144-31 =113\}$. Esto se realiza para cada n-grama.
Definimos la función `calculate_differences(dictionary)` que recibe como parámetro un diccionario de n-gramas y regresa un diccionario, donde cada clave es un n-grama y cada valor es una lista de diferencias.

In [55]:
def calculate_differences(dictionary):
    def list_differences(list_):
        local_differences = []
        for i in range(len(list_)-1):
            for j in range(i+1,len(list_)):
                diff = list_[j] - list_[i]
                if diff not in local_differences:
                    local_differences.append(diff)
        return local_differences
    dict_differences = {}
    for key,value in dictionary.items():
        dict_differences[key] = list_differences(value)
    return dict_differences

# Preprocesado de diferencias
Antes de buscar la longitud de la clave, el diccionario de diferencias pasa por un preprocesado
para que sea más fácil trabajar con las diferencias. Se convierte en una lista y se eliminan los elementos
duplicados. 

In [66]:
def process_differences(dictionary):
    dic_list = list(dictionary.values())
    full_list = []
    for element in dic_list:
        full_list += element
    full_list_unique = list(set(full_list))
    return full_list_unique

# Búsqueda de la longitud de la clave I
En este paso hay varias formas de resolverlo. Debido a la naturaleza del problema, no hay una respuesta que sea válida contundentemente, el proceso de búsqueda de la longitud de la clave está sujeto a probar distintos métodos. Para este paso, se tiene que encontrar un número que divida a la mayoría de las diferencias, una posible solución sería encontrar el *máximo común divisor* (mcd) de la lista de diferencias. Es decir, calcular $gcd(a_1,a_2,..a_n)$. Una forma de realizar este cálculo eficientemente es recordar el teorema de euclides : $gcd(a,b) = gcd(a,b \mod a)$. De igual forma, la función $gcd$ es distributiva y conmutativa, es decir, $gcd(a,gcd(b,c)) = gcd(gcd(a,b),c)$. Con esto en mente, definimos la función `mcd(differences)` que recibe como parámetro una lista de diferencias, y regresa el $gcd$ de los elementos de esa lista.

In [68]:
def mcd(differences):
    def gcd (a,b):
        if (b == 0):
            return a
        else:
             return gcd (b, a % b)
    res = differences[0]
    for c in differences[1::]:
        res = gcd(res , c)
    return res

# Búsqueda de la longitud de la clave II

Calcular el mcd de la lista puede sonar una idea tentadora, pero para el criptoanálisis podría representar una pérdida de información. En lugar de esto, podría realizarse una búsqueda de varias longitudes de clave, encontrar un número que divida a la mayoría de los elementos de la lista (pero no a todos).Sea $A$ la lista de diferencias, así entonces es posible realizar una búsqueda secuencial de posibles longitudes de clave, iniciando desde 1 (el cual dividirá a todos los elementos de la lista) y terminando hasta $ \frac{A_{max}}{2}$ , donde $A_{max} = max\{a_1,a_2,a_3...a_n\}$ el máximo de la lista de diferencias. ¿Por qué $ \frac{A_{max}}{2}$ ? Porque para encontrar los divisores de un número $a$ iniciando desde 1 y siguiendo 2,3... no tiene sentido seguir buscando después de $\frac{a}{2}$, ¡no hay más divisores! Así, se toma como base el máximo de la lista, puesto que $M_1 > M_2 > M_3 ... M_n$ donde $M_i$ i-ésimo valor más grande de$A$, dividido entre 2.
Se define la función `calculate_key_length(differences)` a continuación. Realiza el procedimiento mencionado anteriormente, obteniendo un diccionario de la forma {número:frecuencia} donde frecuencia es el número de elementos que divide de $A$. Por defecto muestra los 3 que mayor frecuencia tienen,pero esto se puede ajustar. 

In [71]:
def calculate_key_length(differences,top=3):
    dict_key_lengths = {}
    start = 2
    end = max(differences)/2
    dict_key_lenghts[1] = len(differences) #1 divide a todos.
    for i in range(2,end+1):
        for element in differences:
            if element%i == 0:
                if i not in dict_key_lengths:
                    dict_key_lengths[i] = 1
                else:
                    dict_key_lengths[i] += 1
    dict_key_lengths = sorted(dict_key_lengths.items(), key=lambda kv: len(kv[1]),reverse=True)
    dict_key_lengths = dict(dict_key_lengths[:top]) 
    return dict_key_lengths
    